In [20]:
import pandas as pd
import numpy as np
import spacy
spacy.load('en_core_web_lg')
import transformers

In [21]:
# from transformers import AutoConfig
transformers.AutoConfig
print("1")
from transformers import FlaxAutoModelForSeq2SeqLM

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Public\Projects\huji_hackathon_22\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ניצן\AppData\Local\Temp\ipykernel_21300\3275634054.py", line 2, in <cell line: 2>
    transformers.AutoConfig
  File "C:\Users\Public\Projects\huji_hackathon_22\venv\lib\site-packages\transformers\__init__.py", line 2188, in __getattr__
    "TFXLNetModel",
  File "C:\Users\Public\Projects\huji_hackathon_22\venv\lib\site-packages\transformers\file_utils.py", line 1490, in __getattr__
  File "C:\Users\Public\Projects\huji_hackathon_22\venv\lib\site-packages\transformers\__init__.py", line 2182, in _get_module
    "TFXLNetForMultipleChoice",
  File "C:\Users\ניצן\AppData\Local\Programs\Python\Python39\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>

In [ ]:
from transformers import AutoTokenizer


In [ ]:
MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

In [ ]:
prefix = "items: "

generation_kwargs = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}


special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}
def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")

    return text

def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]

    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

def generation_function(texts):
    _inputs = texts if isinstance(texts, list) else [texts]
    inputs = [prefix + inp for inp in _inputs]
    inputs = tokenizer(
        inputs,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )
    return generated_recipe
